## Seabed Mobility [2019, 2020, 2023, 2024]

In [5]:
import arcpy
import numpy as np
import os

# Set environment
arcpy.env.overwriteOutput = True
workspace = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin"
arcpy.env.workspace = workspace

# Input rasters (update file paths once 2024 data is available)
raster_2019 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240206_from_WTP_004502963-03\004502963-03-MASP - GIS Data - Task 1 - Seabed Morphology and Mobility\RWE_Thor_Task1_v0.3.gdb\SN2019_011_R_DTM_MBES_100CM"
raster_2020 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240206_from_WTP_004502963-03\004502963-03-MASP - GIS Data - Task 1 - Seabed Morphology and Mobility\RWE_Thor_Task1_v0.3.gdb\MMT_628_GU_OWF_T50R_DTU15_MSL_DTM_100CM"
raster_2023 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\SITEINV\SITEINV.gdb\DK_THO_Fugro_MBES_2023_DEM_Mean_1pt00m_ra_UTM32N_EXT"
raster_2024 = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\1_INPUT\SITEINV\20240924_Bathy\F248460_RWE_Thor_1m_MSL\F248460_RWE_Thor_1m_MSL.tif"


wtg = r"\\WM20ocqu46ph01\WF_Projects\DK_THO\2_FINAL\WTG\WTG.gdb\DK_THO_Layout_pt_UTM32N_v7"

# Buffer parameters
buffer_distance = "200 meters"  # Example buffer size

# Functions

In [7]:
# Function to create a buffer around the WTG locations
def create_wtg_buffer(wtg_fc, buffer_distance, output_buffer):
    arcpy.Buffer_analysis(wtg_fc, output_buffer, buffer_distance)
    print(f"Buffer created around WTG locations: {output_buffer}")

# Function to clip raster to the buffer zone
def clip_raster_to_buffer(input_raster, buffer_fc, output_clipped_raster):
    arcpy.Clip_management(input_raster, "#", output_clipped_raster, buffer_fc, "NoData", "ClippingGeometry")
    print(f"Raster clipped to buffer zone: {output_clipped_raster}")

# Function to calculate raster difference
def calculate_difference_raster(raster1, raster2, output_name, mean_correction=0):
    diff_raster = arcpy.sa.Raster(raster1) - arcpy.sa.Raster(raster2)
    if mean_correction != 0:
        diff_raster += mean_correction
    diff_raster.save(os.path.join(workspace, output_name))
    return diff_raster

# Function to calculate zonal statistics
def calculate_zonal_statistics(diff_raster, buffer_fc, zone_field, output_table):
    # Absolute value raster for mean of magnitude
    abs_diff_raster = arcpy.sa.Abs(diff_raster)

    # Create Zonal Statistics tables for Max, Min, and Mean
    arcpy.sa.ZonalStatisticsAsTable(buffer_fc, zone_field, diff_raster, output_table, "DATA", "ALL")
    
    # Add the field with the new shorter name
    arcpy.AddField_management(output_table, mean_field_name, "DOUBLE")
    
    # Calculate mean of magnitude (average of absolute values)
    abs_table = os.path.join(workspace, "abs_" + os.path.basename(output_table))
    arcpy.sa.ZonalStatisticsAsTable(buffer_fc, zone_field, abs_diff_raster, abs_table, "DATA", "MEAN")
    
    # Ensure the correct field name is referenced
    with arcpy.da.UpdateCursor(output_table, [zone_field, mean_field_name]) as cursor:
        with arcpy.da.SearchCursor(abs_table, [zone_field, "MEAN"]) as abs_cursor:
            abs_dict = {row[0]: row[1] for row in abs_cursor}  # Ensure this field is correct
            for row in cursor:
                zone_id = row[0]
                if zone_id in abs_dict:
                    row[1] = abs_dict[zone_id]
                cursor.updateRow(row)

    print(f"Zonal statistics table created: {output_table}")


## A) Calculate difference rasters B) clip them to the buffer zone C) compute statistics

In [9]:

# Create buffer around WTG locations
wtg_buffer = os.path.join(workspace, "WTG_Buffer_500m.shp")
create_wtg_buffer(wtg, buffer_distance, wtg_buffer)

# List of output difference raster names and correction values
raster_pairs = [
    (raster_2024, raster_2019, "diff_2024_2019_clipped.tif", 0),     # 2024 - 2019
    (raster_2024, raster_2023, "diff_2024_2023_clipped.tif", 0),     # 2024 - 2023
    (raster_2023, raster_2019, "diff_2023_2019_clipped.tif", 0.11),  # 2023 - 2019 (+0.11)
    (raster_2023, raster_2020, "diff_2023_2020_clipped.tif", 0.09),  # 2023 - 2020 (+0.09)
    (raster_2020, raster_2019, "diff_2020_2019_clipped.tif", 0)      # 2020 - 2019 (no correction)
]

# Calculate difference rasters, clip them to the buffer zone, and compute statistics
for r1, r2, output_name, correction in raster_pairs:
    if arcpy.Exists(r1) and arcpy.Exists(r2):  # Only process if both rasters exist
        print(f"Calculating difference for {output_name}...")
        diff_raster = calculate_difference_raster(r1, r2, output_name, correction)
        
        # Clip the difference raster to the WTG buffer zone
        output_clipped_raster = os.path.join(workspace, "clipped_" + output_name)
        clip_raster_to_buffer(diff_raster, wtg_buffer, output_clipped_raster)
        


Buffer created around WTG locations: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\WTG_Buffer_500m.shp
Calculating difference for diff_2024_2019_clipped.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\clipped_diff_2024_2019_clipped.tif
Calculating difference for diff_2024_2023_clipped.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\clipped_diff_2024_2023_clipped.tif
Calculating difference for diff_2023_2019_clipped.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\clipped_diff_2023_2019_clipped.tif
Calculating difference for diff_2023_2020_clipped.tif...
Raster clipped to buffer zone: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\clipped_dif

In [10]:
# Change the field name to a shorter version
mean_field_name = "MeanMag"  # Shorter field name

# Zone field for WTG locations (use the field that represents unique WTG identifiers)
zone_field = "Layout_ID1"

# Calculate difference rasters, clip them to the buffer zone, and compute statistics
for r1, r2, output_name, correction in raster_pairs:
    if arcpy.Exists(r1) and arcpy.Exists(r2):  # Only process if both rasters exist
        print(f"Calculating calculate_zonal_statistics for {output_name}...")
        output_clipped_raster = os.path.join(workspace, "clipped_" + output_name)
        output_table = os.path.join(workspace, "zonal_stats_" + os.path.basename(output_name).replace(".tif", ".dbf"))
        # Calculate zonal statistics and add to attribute table
        calculate_zonal_statistics(output_clipped_raster, wtg_buffer, zone_field, output_table)

Calculating calculate_zonal_statistics for diff_2024_2019_clipped.tif...
Zonal statistics table created: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\zonal_stats_diff_2024_2019_clipped.dbf
Calculating calculate_zonal_statistics for diff_2024_2023_clipped.tif...
Zonal statistics table created: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\zonal_stats_diff_2024_2023_clipped.dbf
Calculating calculate_zonal_statistics for diff_2023_2019_clipped.tif...
Zonal statistics table created: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\zonal_stats_diff_2023_2019_clipped.dbf
Calculating calculate_zonal_statistics for diff_2023_2020_clipped.tif...
Zonal statistics table created: \\WM20ocqu46ph01\WF_Projects\DK_THO\4_OUTPUT\SITEINV\20241010_SeabedMobility_JustinPeterLarkin\zonal_stats_diff_2023_2020_clipped.dbf
Calculating calculate_zonal_statistics for d